In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
my_url = "http://floatingmedia.com/columbia/BBC.html"
raw_html = requests.get(my_url)
soup_doc = BeautifulSoup(raw_html.content, "html.parser")
print(soup_doc.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   BBC - Culture - The 21st Century’s 100 greatest films: Who voted?
  </title>
  <meta content="story, STORY, story, image, the-100-greatest-films-of-the-21st-century, " name="keywords"/>
  <meta content="We polled 177 critics from around the world – here is how they voted." name="description"/>
  <meta content="The 21st Century’s 100 greatest films: Who voted?" property="og:title">
   <meta content="article" property="og:type">
    <meta content="http://www.bbc.com/culture/story/20160819-the-21st-centurys-100-greatest-films-who-voted" property="og:url">
     <meta content="We polled 177 critics from around the world – here is how they voted." property="og:description">
      <meta content="summary_large_image" name="twitter:card"/>
      <meta content="@BBC_Culture" name="twitter:site"/>
      <meta content="The 21st Century’s 100 greatest fil

In [3]:
all_info = soup_doc.find_all('div')

In [4]:
all_info = soup_doc.find_all(class_='body-content')

In [5]:
for foo in all_info:
    all_p = foo.find_all('p')
    
all_p

[<p>Communicating with 177 film critics is a time-consuming process. But for every critic who participated – and many more were invited – it wasn’t just a matter of lending their expertise; it was about sharing their passion. The critics who participated hail from 36 countries: 81 from the US, 19 from the UK, five each from Canada, Cuba, France, and Germany, and four each from Australia, Colombia, India, Israel and Italy. Lebanon, the UAE, China, Bangladesh, Chile, Namibia, Kazakhstan and many others are represented too. Of the 177 critics, 55 are women and 122 are men. We present their votes here in alphabetical order.</p>,
 <p><strong>Simon Abrams – Freelance film critic (US)</strong></p>,
 <p>1. Mulholland Drive (David Lynch, 2001)<br/>2. In the Mood for Love (Wong Kar-wai, 2000)<br/>3. The Tree of Life (Terrence Malick, 2011)<br/>4. Yi Yi: A One and a Two (Edward Yang, 2000)<br/>5. Goodbye to Language (Jean-Luc Godard, 2014)<br/>6. The White Meadows (Mohammad Rasoulof, 2009)<br/>7.

# Getting the critics

In [16]:
list_of_critics = []

for line in all_p:
    critics_info = {}
    if line.strong is not None:
        critic_info = line.strong.string
        regex_for_name = r"(^\w.*) –"
        regex_for_org = r"– (\w.*) \("
        regex_for_cn = r"\((.*)\)"
        critics_info['name'] = re.findall(regex_for_name,critic_info, re.IGNORECASE)
        critics_info['org'] = re.findall(regex_for_org,critic_info, re.IGNORECASE)
        critics_info['cn'] = re.findall(regex_for_cn,critic_info, re.IGNORECASE)
        list_of_critics.append(critics_info)
        

In [19]:
list_of_critics

[{'name': ['Simon Abrams'], 'org': ['Freelance film critic'], 'cn': ['US']},
 {'name': ['Sam Adams'], 'org': ['Freelance film critic'], 'cn': ['US']},
 {'name': ['Thelma Adams'], 'org': ['Freelance film critic'], 'cn': ['US']},
 {'name': ['Arturo Aguilar'],
  'org': ['Rolling Stone Mexico'],
  'cn': ['Mexico']},
 {'name': ['Matthew Anderson'], 'org': ['BBC Culture'], 'cn': ['UK']},
 {'name': ['Tim Appelo'], 'org': ['The Wrap'], 'cn': ['US']},
 {'name': ['Adriano Aprà'], 'org': ['Film historian'], 'cn': ['Italy']},
 {'name': ['Michael Arbeiter'], 'org': ['Nerdist'], 'cn': ['US']},
 {'name': ['Ali Arikan'], 'org': ['Dipnot TV'], 'cn': ['Turkey']},
 {'name': ['Michael Atkinson'], 'org': ['The Village Voice'], 'cn': ['US']},
 {'name': ['Ana Maria Bahiana'],
  'org': ['Freelance film critic'],
  'cn': ['Brazil']},
 {'name': ['Cameron Bailey'],
  'org': ['Toronto Film Festival'],
  'cn': ['Canada']},
 {'name': ['Lindsay Baker'], 'org': ['BBC Culture'], 'cn': ['UK']},
 {'name': ['Miriam Bale'

In [20]:
import pandas as pd

In [21]:
df_critics = pd.DataFrame(list_of_what)

In [23]:
df_critics.to_csv('BBC_critics.csv', index=False)

# Getting the films

In [24]:
list_of_films = []

for line in all_p:
    if line.strong is not None:
        critic_info = line.strong.string
        movie_info = line.next_sibling
        each_movie = movie_info.find_all(string=True)
        for movie in each_movie:
            film_info1 = {}
            regex_for_title = r"^\d\d?\. (.*) \("
            regex_for_director = r"\((.*)\,"
            regex_for_year = r" (\d{4})\)"
            film_info1['title'] = re.findall(regex_for_title, movie, re.IGNORECASE)
            film_info1['director'] = re.findall(regex_for_director, movie, re.IGNORECASE)
            film_info1['year'] = re.findall(regex_for_year, movie, re.IGNORECASE)
            list_of_films.append(film_info1)

In [29]:
df_films = pd.DataFrame(list_of_films)

In [30]:
df_films.to_csv('BBC_directors_films.csv', index = False)

# Aggregating the data and getting the countries by most voted films

In Terminal:

CREATE TABLE bbc_directors_films_2 (director varchar(120), film varchar(500), year int);

\copy bbc_directors_films_2 from /Users/dianabaptistarojo/Documents/Columbia/classwork_summer/data_and_databases/final_project/BBC_directors_films.csv delimiter ',' csv

In [31]:
import pg8000
conn = pg8000.connect(database="dircountries",user="dianabaptistarojo")
print(type(conn))

<class 'pg8000.core.Connection'>


In [ ]:
conn.rollback()

In [51]:
popular_films = []

cursor = conn.cursor()
myquery = "SELECT count(film), film, director FROM bbc_directors_films_2 GROUP BY film, director ORDER BY count(film) DESC LIMIT 100;"

cursor.execute(myquery)
for row in cursor:
    pop_film = row
    popular_films.append(pop_film)

In [52]:
popular_films

[[49, 'In the Mood for Love', 'Wong Kar-wai'],
 [47, 'Mulholland Drive', 'David Lynch'],
 [35, 'There Will Be Blood', 'Paul Thomas Anderson'],
 [34, 'Spirited Away', 'Hayao Miyazaki'],
 [30, 'Boyhood', 'Richard Linklater'],
 [29, 'Eternal Sunshine of the Spotless Mind', 'Michel Gondry'],
 [28, 'A Separation', 'Asghar Farhadi'],
 [23, 'The Tree of Life', 'Terrence Malick'],
 [22, 'Yi Yi: A One and a Two', 'Edward Yang'],
 [21, 'No Country For Old Men', 'Joel and Ethan Coen'],
 [20, 'Inside Llewyn Davis', 'Joel and Ethan Coen'],
 [18, 'Children of Men', 'Alfonso Cuarón'],
 [17, '"Pans Labyrinth"', 'Guillermo Del Toro'],
 [17, '4 Months, 3 Weeks & 2 Days', 'Cristian Mungiu'],
 [16, 'The Act of Killing', 'Joshua Oppenheimer'],
 [16, 'Holy Motors', 'Leos Carax'],
 [15, 'Zodiac', 'David Fincher'],
 [14, 'Mad Max: Fury Road', 'George Miller'],
 [13, 'The Social Network', 'David Fincher'],
 [13, 'Talk to Her', 'Pedro Almodóvar'],
 [13, 'The White Ribbon', 'Michael Haneke'],
 [13, 'Caché', 'Mic

In [53]:
import numpy as np
col_names = ['count', 'film', 'director']
df_filmcount = pd.DataFrame.from_records(popular_films, columns=col_names)
df_filmcount.loc[53]

count                                  7
film        Once Upon a Time in Anatolia
director               Nuri Bilge Ceylan
Name: 53, dtype: object

In [55]:
df_filmcount

,count,film,director
0,49,In the Mood for Love,Wong Kar-wai
1,47,Mulholland Drive,David Lynch
2,35,There Will Be Blood,Paul Thomas Anderson
3,34,Spirited Away,Hayao Miyazaki
4,30,Boyhood,Richard Linklater
5,29,Eternal Sunshine of the Spotless Mind,Michel Gondry
6,28,A Separation,Asghar Farhadi
7,23,The Tree of Life,Terrence Malick
8,22,Yi Yi: A One and a Two,Edward Yang
9,21,No Country For Old Men,Joel and Ethan Coen


In [46]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [47]:
Null_country = "No country recorded."

In [48]:
def get_film_info(row):
    driver = webdriver.Chrome()
    driver.get("https://www.imdb.com/")
    try:
        text_input = driver.find_element_by_xpath('//*[@id="navbar-query"]')
        text_input.send_keys(row['film'])
        button = driver.find_element_by_xpath('//*[@id="navbar-submit-button"]')
        button.click()
        select_film = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a')
        select_film.click()
        select_country = driver.find_element_by_xpath('//*[@id="titleDetails"]/div[2]/a[1]').text
        print(select_country)
        
    except:
        print(Null_country) 
       
    driver.quit()

In [58]:
df_filmcount.apply(get_film_info, axis=1)

Hong Kong
English
USA
Japan
USA
USA
Iran
USA
Mandarin
USA
USA
USA
Spain
Romania
UK
France
English
Australia
USA
Spain
Germany
France
English
USA
USA
USA
USA
USA
USA
South Korea
Germany
USA
Hungary
Germany
USA
Brazil
Thailand
USA
Mauritania
France
USA
No country recorded.
UK
USA
USA
Mandarin
USA
Austria
USA
France
Germany
USA
English
Turkey
France
Denmark
UK
Australia
Thailand
USA
Argentina
Germany
USA
Italy
Hungary
UK
Poland
English
UK
Switzerland
Canada
USA
Russia
France
USA
UK
USA
English
Thailand
USA
France
Netherlands
UK
USA
France
English
France
USA
USA
English
France
USA
China
France
USA
USA
USA
UK
Denmark
USA


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
      ... 
70    None
71    None
72    None
73    None
74    None
75    None
76    None
77    None
78    None
79    None
80    None
81    None
82    None
83    None
84    None
85    None
86    None
87    None
88    None
89    None
90    None
91    None
92    None
93    None
94    None
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object

In [194]:
countries = open("films_top_countries.txt", "r")
film_countries = countries.read()
film_countries

'China\nUnited Kingdom\nUnited States of America\nJapan\nUnited States of America\nUnited States of America\nIran\nUnited States of America\nChina\nUnited States of America\nUnited States of America\nUnited States of America\nSpain\nRomania\nUnited Kingdom\nFrance\nUnited Kingdom\nAustralia\nUnited States of America\nSpain\nGermany\nFrance\nUnited Kingdom\nUnited States of America\nUnited States of America\nUnited States of America\nUnited States of America\nUnited States of America\nUnited States of America\nSouth Korea\nGermany\nUnited States of America\nHungary\nGermany\nUnited States of America\nBrazil\nThailand\nUnited States of America\nMauritania\nFrance\nUnited States of America\nUnited States of America\nUnited Kingdom\nUnited States of America\nUnited States of America\nChina\nUnited States of America\nAustria\nUnited States of America\nFrance\nGermany\nUnited States of America\nUnited Kingdom\nTurkey\nFrance\nDenmark\nUnited Kingdom\nAustralia\nThailand\nUnited States of Ame

In [195]:
countrylines = film_countries.split('\n')
countrylines

['China',
 'United Kingdom',
 'United States of America',
 'Japan',
 'United States of America',
 'United States of America',
 'Iran',
 'United States of America',
 'China',
 'United States of America',
 'United States of America',
 'United States of America',
 'Spain',
 'Romania',
 'United Kingdom',
 'France',
 'United Kingdom',
 'Australia',
 'United States of America',
 'Spain',
 'Germany',
 'France',
 'United Kingdom',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'United States of America',
 'South Korea',
 'Germany',
 'United States of America',
 'Hungary',
 'Germany',
 'United States of America',
 'Brazil',
 'Thailand',
 'United States of America',
 'Mauritania',
 'France',
 'United States of America',
 'United States of America',
 'United Kingdom',
 'United States of America',
 'United States of America',
 'China',
 'United States of America',
 'Austria',
 'United States of Ameri

In [196]:
df_filmcount['country'] = countrylines

In [197]:
df_filmcount

,count,film,director,country
0,49,In the Mood for Love,Wong Kar-wai,China
1,47,Mulholland Drive,David Lynch,United Kingdom
2,35,There Will Be Blood,Paul Thomas Anderson,United States of America
3,34,Spirited Away,Hayao Miyazaki,Japan
4,30,Boyhood,Richard Linklater,United States of America
5,29,Eternal Sunshine of the Spotless Mind,Michel Gondry,United States of America
6,28,A Separation,Asghar Farhadi,Iran
7,23,The Tree of Life,Terrence Malick,United States of America
8,22,Yi Yi: A One and a Two,Edward Yang,China
9,21,No Country For Old Men,Joel and Ethan Coen,United States of America


In [198]:
import numpy as np

In [199]:
df_filmcount.groupby('country')['film'].nunique()

country
Argentina                    1
Australia                    2
Austria                      1
Brazil                       1
Canada                       1
China                        4
Denmark                      2
France                      11
Germany                      5
Hungary                      2
Iran                         1
Italy                        1
Japan                        1
Mauritania                   1
Netherlands                  1
Poland                       1
Romania                      1
Russia                       1
South Korea                  1
Spain                        2
Switzerland                  1
Thailand                     3
Turkey                       1
United Kingdom              16
United States of America    38
Name: film, dtype: int64

In [228]:
df_filmcount2 = df_filmcount.groupby(['country', 'count'])['film'].value_counts().reset_index(name='value')
df_filmcount2.head()



,country,count,film,value
0,Argentina,6,The Headless Woman,1
1,Australia,6,Moulin Rouge!,1
2,Australia,14,Mad Max: Fury Road,1
3,Austria,8,Amour,1
4,Brazil,10,City of God,1


In [238]:
df_filmcount2["string"] = df_filmcount2['film'] + ": " + df_filmcount2['count'].map(str) + np.where(df_filmcount2["count"]>1, ' votes', ' vote')
df_filmcount2.head()

,country,count,film,value,string
0,Argentina,6,The Headless Woman,1,The Headless Woman: 6 votes
1,Australia,6,Moulin Rouge!,1,Moulin Rouge!: 6 votes
2,Australia,14,Mad Max: Fury Road,1,Mad Max: Fury Road: 14 votes
3,Austria,8,Amour,1,Amour: 8 votes
4,Brazil,10,City of God,1,City of God: 10 votes


In [240]:
film_output = df_filmcount2.groupby('country')['string'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='properties.article')
film_output.head()

,country,properties.article
0,Argentina,<p>The Headless Woman: 6 votes</p>
1,Australia,<p>Moulin Rouge!: 6 votes</p><p> Mad Max: Fury...
2,Austria,<p>Amour: 8 votes</p>
3,Brazil,<p>City of God: 10 votes</p>
4,Canada,<p>Stories We Tell: 6 votes</p>


In [245]:
df_filmcount.head()

,count,film,director,country
0,49,In the Mood for Love,Wong Kar-wai,China
1,47,Mulholland Drive,David Lynch,United Kingdom
2,35,There Will Be Blood,Paul Thomas Anderson,United States of America
3,34,Spirited Away,Hayao Miyazaki,Japan
4,30,Boyhood,Richard Linklater,United States of America


In [249]:
dirs = df_filmcount.groupby('country')['film'].nunique().reset_index(name='properties.headline')
film_output2 = film_output.merge(dirs, how='left', on='country')
film_output2.head()

,country,properties.article,properties.headline
0,Argentina,<p>The Headless Woman: 6 votes</p>,1
1,Australia,<p>Moulin Rouge!: 6 votes</p><p> Mad Max: Fury...,2
2,Austria,<p>Amour: 8 votes</p>,1
3,Brazil,<p>City of God: 10 votes</p>,1
4,Canada,<p>Stories We Tell: 6 votes</p>,1


In [250]:
film_output2.rename(columns = {'country': 'properties.name'}, inplace=True)
film_output2.head()

,properties.name,properties.article,properties.headline
0,Argentina,<p>The Headless Woman: 6 votes</p>,1
1,Australia,<p>Moulin Rouge!: 6 votes</p><p> Mad Max: Fury...,2
2,Austria,<p>Amour: 8 votes</p>,1
3,Brazil,<p>City of God: 10 votes</p>,1
4,Canada,<p>Stories We Tell: 6 votes</p>,1


In [251]:
film_output2.to_csv('Film_country_output.csv', index=False)

# Aggregate and get the directors country

In [ ]:
In Terminal:

CREATE TABLE bbc_directors_films_2 (director varchar(120), film varchar(500), year int);

\copy bbc_directors_films_2 from /Users/dianabaptistarojo/Documents/Columbia/classwork_summer/data_and_databases/final_project/BBC_directors_films.csv delimiter ',' csv

In [90]:
popular_directors = []

cursor = conn.cursor()
myquery = "SELECT count(director), director FROM bbc_directors_films_2 GROUP BY director ORDER BY count(director) DESC LIMIT 100;"

cursor.execute(myquery)
for row in cursor:
    pop_director = row
    popular_directors.append(pop_director)

In [92]:
popular_directors

[[52, 'Paul Thomas Anderson'],
 [52, 'Joel and Ethan Coen'],
 [51, 'Wong Kar-wai'],
 [48, 'David Lynch'],
 [39, 'Richard Linklater'],
 [35, 'Michael Haneke'],
 [35, 'Hayao Miyazaki'],
 [32, 'Terrence Malick'],
 [31, 'Asghar Farhadi'],
 [31, 'David Fincher'],
 [30, 'Michel Gondry'],
 [28, 'Wes Anderson'],
 [28, 'Christopher Nolan'],
 [24, 'Alfonso Cuarón'],
 [22, 'Edward Yang'],
 [21, 'Ang Lee'],
 [21, 'Apichatpong Weerasethakul'],
 [20, 'Joshua Oppenheimer'],
 [19, 'Quentin Tarantino'],
 [18, 'Cristian Mungiu'],
 [17, 'Guillermo Del Toro'],
 [16, 'Kenneth Lonergan'],
 [16, 'Leos Carax'],
 [16, 'Pedro Almodóvar'],
 [16, 'Lars von Trier'],
 [15, 'Abbas Kiarostami'],
 [15, 'Nuri Bilge Ceylan'],
 [15, 'Steve McQueen'],
 [15, 'Andrey Zvyagintsev'],
 [15, 'Sofia Coppola'],
 [14, 'Spike Lee'],
 [14, 'George Miller'],
 [13, 'Jia Zhangke'],
 [12, 'Todd Haynes'],
 [12, 'Charlie Kaufman'],
 [12, 'Park Chan-wook'],
 [12, 'Andrew Stanton'],
 [11, 'Kathryn Bigelow'],
 [11, 'Abderrahmane Sissako'],
 

In [109]:
import numpy as np
col_names = ['count', 'director']
df_dircount = pd.DataFrame.from_records(popular_directors, columns=col_names)
df_dircount[60:]

,count,director
60,7,Paolo Sorrentino
61,7,Abdellatif Kechiche
62,7,Jean-Pierre and Luc Dardenne
63,7,Gus Van Sant
64,7,Harmony Korine
65,7,Jim Jarmusch
66,6,Baz Luhrmann
67,6,David Cronenberg
68,6,Paweł Pawlikowski
69,6,Werner Herzog


In [95]:
def get_dir_info(row):
    driver = webdriver.Chrome()
    driver.get("https://www.imdb.com/")
    try:
        text_input = driver.find_element_by_xpath('//*[@id="navbar-query"]')
        text_input.send_keys(row['director'])
        button = driver.find_element_by_xpath('//*[@id="navbar-submit-button"]')
        button.click()
        select_director = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a')
        select_director.click()
        select_country = driver.find_element_by_xpath('//*[@id="name-born-info"]/a').text
        print(select_country)
        
    except:
        print(Null_country) 
       
    driver.quit()

In [98]:
df_dircount[82:100].apply(get_dir_info, axis=1)

Brunico, Trentino-Alto Adige, Italy
No country recorded.
Fremont, California, USA
Podorvikha, Irkutskaya oblast, RSFSR, USSR [now Russia]
Brooklyn, New York City, New York, USA
Brooklyn, New York, USA
Mianeh, Azarbaijan Province, Iran
Palm Springs, California, USA
No country recorded.
Buenos Aires, Argentina
Lidingö, Stockholms län, Sweden
Ziguenchor, Casamance, Senegal
Santiago, Chile
Paris, France
Bucharest, Romania
Kobe, Japan
Oakland, California, USA
Brussels, Belgium


82    None
83    None
84    None
85    None
86    None
87    None
88    None
89    None
90    None
91    None
92    None
93    None
94    None
95    None
96    None
97    None
98    None
99    None
dtype: object

In [141]:
countries = open("dir_country.txt", "r")
dir_countries = countries.read()
dir_countries

'Studio City, California, United States of America\nUnited States of America\nShanghai, China\nMissoula, Montana, United States of America\nHouston, Texas, United States of America\nMunich, Bavaria, Germany\nTokyo, Japan\nOttawa, Illinois, United States of America\nKhomeyni Shahr, Isfahan, Iran\nDenver, Colorado, United States of America\nVersailles, Seine-et-Oise [now Yvelines], France\nHouston, Texas, United States of America\nLondon, England, United Kingdom\nMexico City, Distrito Federal, Mexico\nShanghai, China\nPingtung, Taiwan\nBangkok, Thailand\nTexas, United States of America\nKnoxville, Tennessee, United States of America\nIasi, Romania\nGuadalajara, Jalisco, Mexico\nNew York City, New York, United States of America\nSuresnes, Seine [now Hauts-de-Seine], France\nCalzada de Calatrava, Ciudad Real, Castilla-La Mancha, Spain\nCopenhagen, Denmark\nTehran, Iran\nIstanbul, Turkey\nBeech Grove, Indiana, United States of America\nNovosibirsk, Novosibirskaya oblast, RSFSR, Russia\nNew 

In [142]:
countrylines = dir_countries.split('\n')
countrylines

['Studio City, California, United States of America',
 'United States of America',
 'Shanghai, China',
 'Missoula, Montana, United States of America',
 'Houston, Texas, United States of America',
 'Munich, Bavaria, Germany',
 'Tokyo, Japan',
 'Ottawa, Illinois, United States of America',
 'Khomeyni Shahr, Isfahan, Iran',
 'Denver, Colorado, United States of America',
 'Versailles, Seine-et-Oise [now Yvelines], France',
 'Houston, Texas, United States of America',
 'London, England, United Kingdom',
 'Mexico City, Distrito Federal, Mexico',
 'Shanghai, China',
 'Pingtung, Taiwan',
 'Bangkok, Thailand',
 'Texas, United States of America',
 'Knoxville, Tennessee, United States of America',
 'Iasi, Romania',
 'Guadalajara, Jalisco, Mexico',
 'New York City, New York, United States of America',
 'Suresnes, Seine [now Hauts-de-Seine], France',
 'Calzada de Calatrava, Ciudad Real, Castilla-La Mancha, Spain',
 'Copenhagen, Denmark',
 'Tehran, Iran',
 'Istanbul, Turkey',
 'Beech Grove, Indiana,

In [143]:
df_dircount['country'] = countrylines

In [144]:
df_dircount.head()

,count,director,country
0,52,Paul Thomas Anderson,"Studio City, California, United States of America"
1,52,Joel and Ethan Coen,United States of America
2,51,Wong Kar-wai,"Shanghai, China"
3,48,David Lynch,"Missoula, Montana, United States of America"
4,39,Richard Linklater,"Houston, Texas, United States of America"


In [147]:
df_dircount['country'] = df_dircount['country'].str.extract('(\w+? ?\w+? ?\w+? ?\w+?$)')
df_dircount.head()

,count,director,country
0,52,Paul Thomas Anderson,United States of America
1,52,Joel and Ethan Coen,United States of America
2,51,Wong Kar-wai,China
3,48,David Lynch,United States of America
4,39,Richard Linklater,United States of America


In [149]:
df_dircount.to_csv('director_country.csv', index=False)

In [150]:
df_dircount.groupby('country')['director'].nunique()

country
Argentina                    2
Australia                    2
Belgium                      2
Brazil                       1
Canada                       2
Chile                        1
China                        4
Denmark                      1
France                       8
Germany                      5
Hungary                      2
Iran                         3
Ireland                      1
Israel                       1
Italy                        2
Japan                        2
Mauritania                   1
Mexico                       3
Poland                       1
Portugal                     1
Romania                      2
Russia                       2
Senegal                      1
South Korea                  4
Spain                        1
Sweden                       2
Taiwan                       1
Thailand                     1
Tunisia                      1
Turkey                       1
United Kingdom               4
United States of America    35


In [263]:
df_dircount2 = df_dircount.groupby(['country', 'count'])['director'].value_counts().reset_index(name='value')
df_dircount2.head()


,country,count,director,value
0,Argentina,4,Gaspar Noé,1
1,Argentina,8,Lucrecia Martel,1
2,Australia,6,Baz Luhrmann,1
3,Australia,14,George Miller,1
4,Belgium,4,Agnès Varda,1


In [264]:
df_dircount2["string"] = df_dircount2['director'] + ": " + df_dircount['count'].map(str) + np.where(df_dircount["count"]>1, ' votes', ' vote')
df_dircount2.head()


,country,count,director,value,string
0,Argentina,4,Gaspar Noé,1,Gaspar Noé: 52 votes
1,Argentina,8,Lucrecia Martel,1,Lucrecia Martel: 52 votes
2,Australia,6,Baz Luhrmann,1,Baz Luhrmann: 51 votes
3,Australia,14,George Miller,1,George Miller: 48 votes
4,Belgium,4,Agnès Varda,1,Agnès Varda: 39 votes


In [265]:
dir_output = df_dircount2.groupby('country')['string'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='properties.article')
dir_output.head()


,country,properties.article
0,Argentina,<p>Gaspar Noé: 52 votes</p><p> Lucrecia Martel...
1,Australia,<p>Baz Luhrmann: 51 votes</p><p> George Miller...
2,Belgium,<p>Agnès Varda: 39 votes</p><p> Jean-Pierre an...
3,Brazil,<p>Fernando Meirelles and Kátia Lund: 35 votes...
4,Canada,<p>David Cronenberg: 32 votes</p><p> Sarah Pol...


In [266]:
df_dircount.head()

,count,director,country
0,52,Paul Thomas Anderson,United States of America
1,52,Joel and Ethan Coen,United States of America
2,51,Wong Kar-wai,China
3,48,David Lynch,United States of America
4,39,Richard Linklater,United States of America


In [270]:
direcs = df_dircount.groupby('country')['director'].nunique().reset_index(name='properties.headline')
dir_output2 = dir_output.merge(direcs, how='left', on='country')
dir_output2.head()

,country,properties.article,properties.headline
0,Argentina,<p>Gaspar Noé: 52 votes</p><p> Lucrecia Martel...,2
1,Australia,<p>Baz Luhrmann: 51 votes</p><p> George Miller...,2
2,Belgium,<p>Agnès Varda: 39 votes</p><p> Jean-Pierre an...,2
3,Brazil,<p>Fernando Meirelles and Kátia Lund: 35 votes...,1
4,Canada,<p>David Cronenberg: 32 votes</p><p> Sarah Pol...,2


In [273]:
dir_output2.rename(columns = {'country': 'properties.name'}, inplace=True)
dir_output2.head()

,properties.name,properties.article,properties.headline
0,Argentina,<p>Gaspar Noé: 52 votes</p><p> Lucrecia Martel...,2
1,Australia,<p>Baz Luhrmann: 51 votes</p><p> George Miller...,2
2,Belgium,<p>Agnès Varda: 39 votes</p><p> Jean-Pierre an...,2
3,Brazil,<p>Fernando Meirelles and Kátia Lund: 35 votes...,1
4,Canada,<p>David Cronenberg: 32 votes</p><p> Sarah Pol...,2


In [274]:
dir_output2.to_csv('dir_output.csv', index=False)

# Clean up the critics data

In [172]:
df_critics2 = pd.read_csv('BBC_critics.csv')

In [173]:
df_critics2.head()

,cn,name,org
0,United States of America,Simon Abrams,Freelance film critic
1,United States of America,Sam Adams,Freelance film critic
2,United States of America,Thelma Adams,Freelance film critic
3,Mexico,Arturo Aguilar,Rolling Stone Mexico
4,United Kingdom,Matthew Anderson,BBC Culture


In [174]:
df_critics2.groupby('cn')['name'].nunique()

cn
Argentina                    2
Australia                    4
Austria                      2
Bangladesh                   1
Belgium                      1
Brazil                       1
Canada                       5
Chile                        2
China                        1
Colombia                     4
Cuba                         5
Egypt                        1
France                       5
Germany                      5
Hong Kong                    1
India                        5
Indonesia                    1
Israel                       4
Italy                        4
Japan                        1
Kazakhstan                   1
Lebanon                      3
Mexico                       2
Namibia                      1
Philippines                  1
Qatar                        1
Senegal                      1
Singapore                    2
South Africa                 1
South Korea                  2
Switzerland                  1
Taiwan                       1
Turke

In [175]:
df_critics3 = df_critics2.groupby('cn')['name'].value_counts().reset_index(name='count')
df_critics3.head()

,cn,name,count
0,Argentina,Diego Batlle,1
1,Argentina,Javier Porta Fouz,1
2,Australia,Adrian Martin,1
3,Australia,David Stratton,1
4,Australia,Jake Wilson,1


In [178]:
df_critics3["string"] = df_critics3['name'] + ": " + df_critics2['org'].map(str) 
df_critics3.head()

,cn,name,count,string
0,Argentina,Diego Batlle,1,Diego Batlle: Freelance film critic
1,Argentina,Javier Porta Fouz,1,Javier Porta Fouz: Freelance film critic
2,Australia,Adrian Martin,1,Adrian Martin: Freelance film critic
3,Australia,David Stratton,1,David Stratton: Rolling Stone Mexico
4,Australia,Jake Wilson,1,Jake Wilson: BBC Culture


In [302]:
critics_output = df_critics3.groupby('cn')['string'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='properties.article')
critics_output.head()

,cn,properties.article
0,Argentina,<p>Diego Batlle: Freelance film critic</p><p> ...
1,Australia,<p>Adrian Martin: Freelance film critic</p><p>...
2,Austria,<p>Alexander Horwath: Film historian</p><p> Do...
3,Bangladesh,<p>Zakir Hossein Raju: Dipnot TV</p>
4,Belgium,<p>Fernand Denis: The Village Voice</p>


In [303]:
critics = df_critics2.groupby('cn')['name'].nunique().reset_index(name='properties.headline')
critics_output2 = critics_output.merge(critics, how='left', on='cn')
critics_output2.head()


,cn,properties.article,properties.headline
0,Argentina,<p>Diego Batlle: Freelance film critic</p><p> ...,2
1,Australia,<p>Adrian Martin: Freelance film critic</p><p>...,4
2,Austria,<p>Alexander Horwath: Film historian</p><p> Do...,2
3,Bangladesh,<p>Zakir Hossein Raju: Dipnot TV</p>,1
4,Belgium,<p>Fernand Denis: The Village Voice</p>,1


In [304]:
critics_output2.rename(columns = {'cn': 'properties.name'}, inplace=True)
critics_output2

,properties.name,properties.article,properties.headline
0,Argentina,<p>Diego Batlle: Freelance film critic</p><p> ...,2
1,Australia,<p>Adrian Martin: Freelance film critic</p><p>...,4
2,Austria,<p>Alexander Horwath: Film historian</p><p> Do...,2
3,Bangladesh,<p>Zakir Hossein Raju: Dipnot TV</p>,1
4,Belgium,<p>Fernand Denis: The Village Voice</p>,1
5,Brazil,<p>Ana Maria Bahiana: Freelance film critic</p>,1
6,Canada,<p>Calum Marsh: Toronto Film Festival</p><p> C...,5
7,Chile,<p>Antonio Martínez: Positif</p><p> Ernesto Ga...,2
8,China,<p>Raymond Zhou: BBC Culture</p>,1
9,Colombia,<p>Juan Carlos González: African Film Festival...,4


In [307]:
critics_output2.to_csv('critics_output.csv', index=False)

In [308]:
critics_output3 = pd.read_csv('critics_output.csv')

In [309]:
critics_output3

,properties.name,properties.article,properties.headline
0,Argentina,<p>Diego Batlle: Freelance film critic</p><p> ...,2
1,Australia,<p>Adrian Martin: Freelance film critic</p><p>...,4
2,Austria,<p>Alexander Horwath: Film historian</p><p> Do...,2
3,Bangladesh,<p>Zakir Hossein Raju: Dipnot TV</p>,1
4,Belgium,<p>Fernand Denis: The Village Voice</p>,1
5,Brazil,<p>Ana Maria Bahiana: Freelance film critic</p>,1
6,Canada,<p>Calum Marsh: Toronto Film Festival</p><p> C...,5
7,Chile,<p>Antonio Martínez: Positif</p><p> Ernesto Ga...,2
8,China,<p>Raymond Zhou: BBC Culture</p>,1
9,Colombia,<p>Juan Carlos González: African Film Festival...,4


# Map the results

In [184]:
import requests
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [310]:
with open('custom.geo.json') as json_data:
    geometry_data = json.load(json_data)
     
for feat in geometry_data['features']:
    del feat["properties"]['brk_group']   
    del feat["properties"]['formal_fr']
    del feat["properties"]['note_adm0']
    del feat["properties"]['note_brk']
    del feat["properties"]['name_alt']
    del feat["properties"]['fips_10']
    del feat["properties"]['formal_en']

In [311]:
out_geometry = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')

In [312]:
out_geometry

,geometry.coordinates,geometry.type,properties.abbrev,properties.abbrev_len,properties.adm0_a3,properties.adm0_a3_is,properties.adm0_a3_un,properties.adm0_a3_us,properties.adm0_a3_wb,properties.adm0_dif,...,properties.subregion,properties.subunit,properties.tiny,properties.type,properties.un_a3,properties.wb_a2,properties.wb_a3,properties.wikipedia,properties.woe_id,type
0,"[[[[-77.53466, 23.75975], [-77.78, 23.71], [-7...",MultiPolygon,Bhs.,4,BHS,BHS,-99,BHS,-99,0,...,Caribbean,The Bahamas,-99,Sovereign country,044,BS,BHS,-99,-99,Feature
1,"[[[[-63.6645, 46.55001], [-62.9393, 46.41587],...",MultiPolygon,Can.,4,CAN,CAN,-99,CAN,-99,0,...,Northern America,Canada,-99,Sovereign country,124,CA,CAN,-99,-99,Feature
2,"[[[-89.14308041050332, 17.80831899664932], [-8...",Polygon,Belize,6,BLZ,BLZ,-99,BLZ,-99,0,...,Central America,Belize,-99,Sovereign country,084,BZ,BLZ,-99,-99,Feature
3,"[[[-82.96578304719736, 8.225027980985985], [-8...",Polygon,C.R.,4,CRI,CRI,-99,CRI,-99,0,...,Central America,Costa Rica,-99,Sovereign country,188,CR,CRI,-99,-99,Feature
4,"[[[-82.26815121125706, 23.188610744717703], [-...",Polygon,Cuba,4,CUB,CUB,-99,CUB,-99,0,...,Caribbean,Cuba,-99,Sovereign country,192,CU,CUB,-99,-99,Feature
5,"[[[-71.71236141629296, 19.714455878167357], [-...",Polygon,Dom. Rep.,9,DOM,DOM,-99,DOM,-99,0,...,Caribbean,Dominican Republic,-99,Sovereign country,214,DO,DOM,-99,-99,Feature
6,"[[[-46.76379, 82.62796], [-43.40644, 83.22516]...",Polygon,Grlnd.,6,GRL,GRL,-99,GRL,-99,1,...,Northern America,Greenland,-99,Country,304,GL,GRL,-99,-99,Feature
7,"[[[-90.09555457229098, 13.735337632700734], [-...",Polygon,Guat.,5,GTM,GTM,-99,GTM,-99,0,...,Central America,Guatemala,4,Sovereign country,320,GT,GTM,-99,-99,Feature
8,"[[[-73.18979061551762, 19.915683905511912], [-...",Polygon,Haiti,5,HTI,HTI,-99,HTI,-99,0,...,Caribbean,Haiti,-99,Sovereign country,332,HT,HTI,-99,-99,Feature
9,"[[[-87.31665442579549, 12.984685777229004], [-...",Polygon,Hond.,5,HND,HND,-99,HND,-99,0,...,Central America,Honduras,-99,Sovereign country,340,HN,HND,-99,-99,Feature


In [313]:
output_complete = pd.merge(critics_output3, out_geometry[['geometry.coordinates','geometry.type','properties.subunit']], left_on='properties.name', right_on='properties.subunit', how='left')

In [314]:
output_complete.head()

,properties.name,properties.article,properties.headline,geometry.coordinates,geometry.type,properties.subunit
0,Argentina,<p>Diego Batlle: Freelance film critic</p><p> ...,2,"[[[[-65.5, -55.2], [-66.45, -55.25], [-66.9599...",MultiPolygon,Argentina
1,Australia,<p>Adrian Martin: Freelance film critic</p><p>...,4,"[[[[145.39797814349484, -40.79254851660589], [...",MultiPolygon,Australia
2,Austria,<p>Alexander Horwath: Film historian</p><p> Do...,2,"[[[16.979666782304037, 48.123497015976305], [1...",Polygon,Austria
3,Bangladesh,<p>Zakir Hossein Raju: Dipnot TV</p>,1,"[[[92.67272098182556, 22.041238918541254], [92...",Polygon,Bangladesh
4,Belgium,<p>Fernand Denis: The Village Voice</p>,1,"[[[3.314971144228537, 51.345780951536085], [4....",Polygon,Belgium


In [315]:
ok_json = json.loads(output_complete.to_json(orient='records'))

In [316]:
def process_to_geojson(file):
    geo_data3 = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data3['features'].append(this_dict)
    return geo_data3

In [317]:
geo_format3 = process_to_geojson(ok_json)

In [318]:
with open('geo-data3.js', 'w') as outfile:
    outfile.write("var infoData = ")
with open('geo-data3.js', 'a') as outfile:
    json.dump(geo_format3, outfile)